In [1]:
import yaml
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
from Explanations_Models.DT_LIME.LIME import LIME
yaml.add_constructor("!join", join)
config = yaml.load(open("config_envs/cartpole.yml"), Loader= yaml.FullLoader)
config.update(yaml.load(open("config_explanations/cartpole.yml"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights()

device:  cpu


In [2]:
limemod = LIME(config, Runner)

In [3]:
limemod.train()

Dist: 0's / 1's 506 494


c:\Users\blav1\anaconda3\envs\XAI_RL\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\DT_LIME\sampling_methods.py:47: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  return torch.tensor(samples)[samples_indicies]


In [4]:
tree = limemod.surr_model.model

In [5]:
from sklearn.tree import plot_tree
fig = plt.figure(figsize=((25,20)))
plot_tree(tree, 
          feature_names = ["x", "vel", "angle", "angle_vel"],
          class_names = ["left", "right"],
          impurity=False,
          proportion=True,
          filled=True
          )
fig.savefig("daTreeMan.png")

In [6]:
limemod.percent_Correct()


Percent Correct: 0.93%


0.93

In [7]:
limemod.absolute_distance()

MSE:  0.02308327
MAE:  0.07835207


c:\Users\blav1\anaconda3\envs\XAI_RL\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\blav1\anaconda3\envs\XAI_RL\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [8]:
Runner.recorder(model = limemod.surr_model, filename = "VIDEOTESTER")

c:\Users\blav1\anaconda3\envs\XAI_RL\lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\VIDEOTESTER folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
c:\Users\blav1\anaconda3\envs\XAI_RL\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Moviepy - Building video c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\VIDEOTESTER\rl-video-step-0.mp4.
Moviepy - Writing video c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\VIDEOTESTER\rl-video-step-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\VIDEOTESTER\rl-video-step-0.mp4
